In [1]:
"""
1호선 열차운행시각표 데이터 - CSV를 SQLite DB로 변환
작성일: 2025-01-09
목적: CSV 파일을 SQLite 데이터베이스로 변환하여 효율적인 데이터 관리 및 조회
"""

'\n1호선 열차운행시각표 데이터 - CSV를 SQLite DB로 변환\n작성일: 2025-01-09\n목적: CSV 파일을 SQLite 데이터베이스로 변환하여 효율적인 데이터 관리 및 조회\n'

In [2]:
# =============================================================================
# 📦 1. 라이브러리 임포트
# =============================================================================
"""
필요한 라이브러리:
- pandas: 데이터 분석 및 CSV 읽기/쓰기
- sqlite3: SQLite 데이터베이스 연결 및 조작
"""
import pandas as pd      # 데이터프레임 처리용 (CSV → DataFrame)
import sqlite3           # SQLite 데이터베이스 연결 (DataFrame → DB 저장)

In [3]:
# =============================================================================
# 📂 2. CSV 파일 읽기 및 데이터 확인
# =============================================================================
"""
CSV 파일을 pandas DataFrame으로 읽어오기
- 파일 경로: r'data\파일명.csv'
   * r'' (raw string): 백슬래시(\)를 이스케이프 문자로 처리하지 않음
   * Windows 경로에서 필수!

- encoding='euc-kr': 한글 인코딩 지정
   * 서울교통공사 공공데이터는 euc-kr 인코딩 사용
   * 안 쓰면 UnicodeDecodeError 발생 가능
"""

# CSV 파일 경로 지정
csv_file_path = r'data\(1호선)서울교통공사_서울 도시철도 열차운행시각표_20250704.csv'

# pandas read_csv()로 파일 읽기
df = pd.read_csv(
   csv_file_path,          # 파일 경로
   encoding='euc-kr'       # 한글 인코딩 (필수!)
)

# 읽은 데이터의 크기 확인 및 출력
# df.shape: (행 개수, 열 개수) 튜플 반환
print(f"✅ CSV 읽기 완료: {df.shape}")
print(f"   - 총 {df.shape[0]:,}행 (데이터 레코드)")  # 행 개수 (쉼표로 천 단위 구분)
print(f"   - 총 {df.shape[1]}열 (컬럼)")              # 열 개수

✅ CSV 읽기 완료: (88431, 12)
   - 총 88,431행 (데이터 레코드)
   - 총 12열 (컬럼)


<>:7: SyntaxWarning: "\)" is an invalid escape sequence. Such sequences will not work in the future. Did you mean "\\)"? A raw string is also an option.
<>:7: SyntaxWarning: "\)" is an invalid escape sequence. Such sequences will not work in the future. Did you mean "\\)"? A raw string is also an option.
C:\Users\Administrator\AppData\Local\Temp\ipykernel_13792\1389221407.py:7: SyntaxWarning: "\)" is an invalid escape sequence. Such sequences will not work in the future. Did you mean "\\)"? A raw string is also an option.
  * r'' (raw string): 백슬래시(\)를 이스케이프 문자로 처리하지 않음


In [4]:
# =============================================================================
# 💾 3. SQLite 데이터베이스로 저장
# =============================================================================
"""
DataFrame을 SQLite DB 테이블로 저장하는 과정

SQLite란?
- 파일 기반 경량 데이터베이스 (별도 서버 설치 불필요)
- .db 파일 하나에 모든 데이터가 저장됨
- SQL 쿼리로 빠르고 효율적인 데이터 조회 가능

Context Manager (with 문) 사용 이유:
- 자동으로 DB 연결을 열고 닫음 (conn.close() 불필요)
- 에러 발생 시에도 안전하게 연결 종료
- 파일 잠김 문제 방지
"""

# DB 파일명 지정 (현재 디렉토리에 생성됨)
db_file = 'metro.db'

# Context Manager로 DB 연결 (권장 방법 ⭐)
with sqlite3.connect(db_file) as conn:
   """
   sqlite3.connect(파일명):
   - 파일이 없으면 자동으로 생성
   - 있으면 기존 DB 파일에 연결
   """
    
   # DataFrame을 SQL 테이블로 저장
   df.to_sql(
      'Metro_Line_1_Schedule',    # 테이블 이름 (SQL에서 사용할 이름)
      conn,                        # DB 연결 객체
      if_exists='replace',         # 기존 테이블이 있으면 삭제 후 재생성
      index=False                  # pandas 인덱스(0,1,2...)는 저장하지 않음
   )
    
   """
   if_exists 옵션 설명:
   - 'fail': 테이블이 이미 있으면 에러 발생 (기본값)
   - 'replace': 기존 테이블 삭제 후 새로 생성 (덮어쓰기)
   - 'append': 기존 테이블에 데이터 추가 (누적)
   
   index=False 설명:
   - True: pandas의 인덱스(0,1,2...)도 컬럼으로 저장
   - False: 인덱스 제외하고 실제 데이터만 저장 (권장)
   """

# with 문 종료 시 자동으로 conn.close() 실행됨
print(f"✅ DB 저장 완료: {db_file}")
print(f"   - 테이블명: Metro_Line_1_Schedule")
print(f"   - 저장된 데이터: {len(df):,}행")

✅ DB 저장 완료: metro.db
   - 테이블명: Metro_Line_1_Schedule
   - 저장된 데이터: 88,431행


In [5]:

# =============================================================================
# 👀 4. 데이터 미리보기
# =============================================================================
"""
주피터 노트북에서 실행 시:
- 셀 마지막에 df.head()만 작성하면 자동으로 HTML 테이블로 표시
- print() 없이도 예쁜 표 형태로 출력됨

df.head() 메서드:
- 기본: 처음 5개 행 반환
- df.head(10): 처음 10개 행 반환
- df.tail(5): 마지막 5개 행 반환
"""

# 처음 5개 행을 HTML 테이블로 출력 (주피터 노트북)
df.head()

,고유번호,호선,역사코드,역사명,주중주말,방향,급행여부,열차코드,열차도착시간,열차출발시간,출발역,도착역
0,1,1,150,서울역,DAY,DOWN,0,K101,12:53:00,12:53:30,양주,인천
1,2,1,150,서울역,DAY,DOWN,0,K103,13:02:30,13:03:00,소요산,인천
2,3,1,150,서울역,DAY,DOWN,0,K107,13:25:00,13:25:30,의정부,인천
3,4,1,150,서울역,DAY,DOWN,0,K11,5:30:00,5:30:30,광운대,인천
4,5,1,150,서울역,DAY,DOWN,0,K111,13:48:00,13:48:30,의정부,인천


In [6]:
# =============================================================================
# 📊 5. (선택사항) 저장된 데이터 검증
# =============================================================================
"""
DB에 데이터가 제대로 저장되었는지 확인하는 코드
주석 해제하여 실행 가능
"""

# # DB에서 데이터 다시 읽어오기
# with sqlite3.connect(db_file) as conn:
#     # SQL 쿼리로 저장된 데이터 확인
#     query = "SELECT COUNT(*) as 총개수 FROM Metro_Line_1_Schedule"
#     result = pd.read_sql(query, conn)
#     print(f"\n✅ DB 검증:")
#     print(f"   - 저장된 행 개수: {result['총개수'][0]:,}개")

#     # 처음 5개 행 조회
#     query2 = "SELECT * FROM Metro_Line_1_Schedule LIMIT 5"
#     sample_data = pd.read_sql(query2, conn)
#     print("\n   - 샘플 데이터:")
#     print(sample_data)

'\nDB에 데이터가 제대로 저장되었는지 확인하는 코드\n주석 해제하여 실행 가능\n'

In [7]:
# =============================================================================
# 💡 추가 팁 및 참고사항
# =============================================================================
"""
1. 파일 경로 작성법 (Windows):
   ✅ r'data\파일명.csv'         # Raw string (권장)
   ✅ 'data/파일명.csv'           # 슬래시 사용
   ✅ 'data\\파일명.csv'          # 백슬래시 이스케이프
   ❌ 'data\파일명.csv'           # 에러 발생!

2. 인코딩 문제 해결:
   - euc-kr 실패 시 → 'cp949' 시도
   - UTF-8 파일 → encoding='utf-8'

3. 대용량 파일 처리:
   - 청크 단위 읽기: pd.read_csv(..., chunksize=10000)
   - 특정 컬럼만: usecols=['컬럼1', '컬럼2']

4. DB 쿼리 예시:
   - 전체 조회: SELECT * FROM Metro_Line_1_Schedule
   - 조건 필터: SELECT * FROM Metro_Line_1_Schedule WHERE 역사명='서울역'
   - 집계: SELECT 역사명, COUNT(*) FROM Metro_Line_1_Schedule GROUP BY 역사명

5. 에러 해결:
   - OperationalError: DB 파일 권한 확인
   - UnicodeDecodeError: encoding 확인
   - FileNotFoundError: 파일 경로 확인
"""


# =============================================================================
# 🎯 코드 실행 요약
# =============================================================================
"""
이 코드는 다음을 수행합니다:
1. CSV 파일 읽기 (88,431행 × 12열)
2. SQLite DB로 변환 (metro.db 파일 생성)
3. Metro_Line_1_Schedule 테이블 생성
4. 데이터 미리보기

결과:
✅ CSV (5.4MB) → DB (6MB)
✅ SQL 쿼리로 빠른 조회 가능
✅ 데이터 무결성 보장
"""

'\n이 코드는 다음을 수행합니다:\n1. CSV 파일 읽기 (88,431행 × 12열)\n2. SQLite DB로 변환 (metro.db 파일 생성)\n3. Metro_Line_1_Schedule 테이블 생성\n4. 데이터 미리보기\n\n결과:\n✅ CSV (5.4MB) → DB (6MB)\n✅ SQL 쿼리로 빠른 조회 가능\n✅ 데이터 무결성 보장\n'